Generate text based on the provided prompt, which is related to identifying causality between medical conditions. The function replaces a placeholder in the prompt with the actual test case, and the generated text is printed and returned. Finally, the function is called once with a specific test case ("A: heart rate, B: sepsis").

Identifying which medical tests, markers, and parameters commonly encountered in clinical settings are related to sepsis. 

# TODO
* get label and fluid
* merge these two together into DF
* add all admission data too?
* match only word

* For the LABEVENTS table, we have to store both the label value and the fluid or the cateogory value. For the Microbiology events, what distinguishes the different ITEMIDs?

Run `rph.sh` script to SSH into server.

# Identifying causality

In [42]:
import pandas as pd
import numpy as np

#datetime calculations
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import requests
import re

# For local streaming, the websockets are hosted without ssl - http://
HOST = 'localhost:5000'
URI = f'http://{HOST}/api/v1/generate'

# For reverse-proxied streaming, the remote will likely host with ssl - https://
# URI = 'https://your-uri-here.trycloudflare.com/api/v1/generate'


def run(prompt,test):
    request = {
        'prompt': prompt.replace("{test}",test),
        'max_new_tokens': 250,
        'temperature': 0.0,
        'top_p': 0.1,
        'length_penalty': 5,
        'early_stopping': True,
        'seed': 0,
    }

    response = requests.post(URI, json=request)
    print(response.json()['results'][0]['text'])

    return response.json()['results'][0]['text']


In [ ]:
prompt_causality = '''
[INST]
You are a medical expert. You are tasked to identify causality between two conditions. 
Return yes if A causes B. If B causes A or if you are not sure, return no. 
Keep your responses short and succinct. 
Input
{test}
[/INST]
'''

We use the local language model to identify causality. Specifically, we are identifying what symptoms are causing sepsis.

In [12]:
# Function to get the most recent value for a given item ID list from a DataFrame
def get_most_recent_value(df, itemids):

    filtered_df = df[df['itemid'].isin(itemids)].sort_values('charttime')

    if not filtered_df.empty:
        return filtered_df.iloc[-1]['value']
    
    else:
        return None

In [5]:
# initialise a dataframe with symptoms
symptom = ['uti', 'low oxygen', 'low neutrophils', 'low BP', 'high BP', 'age>50', 'pregnancy', 'age>70']
df = pd.DataFrame(symptom, columns=['symptom'])
df

,symptom
0,uti
1,low oxygen
2,low neutrophils
3,low BP
4,high BP
5,age>50
6,pregnancy
7,age>70


In [6]:
# get relation by checking symptoms against LLM
df['relation'] = df.apply(lambda x: run(prompt_causality, f"A: {x['symptom']}, B: sepsis"), axis=1)

Yes, A (uti) causes B (sepsis).
Yes, A (low oxygen) causes B (sepsis). Low oxygen levels can lead to sepsis, a life-threatening condition that can cause organ failure and death.
Yes, A (low neutrophils) causes B (sepsis). Low neutrophil counts can impair the body's ability to fight off infections, which can lead to the development of sepsis.
No. Sepsis can cause low blood pressure (BP), but low BP does not cause sepsis.
Yes, A (high BP) can cause B (sepsis). High blood pressure can lead to damage to blood vessels, which can increase the risk of bacterial infection and sepsis.
Yes, A (age > 50) causes B (sepsis). As people age, their immune systems weaken, making them more susceptible to infections like sepsis.
No. Sepsis can be caused by various factors, including infection, but pregnancy is not a known cause of sepsis.
Yes, A (age > 70) causes B (sepsis). As people age, their immune systems weaken, making them more susceptible to infections like sepsis.


In [7]:
# create short result of relation
df['relation_short'] = df['relation'].apply(lambda x: 'yes' if 'yes' in x.lower() else 'no')

In [8]:
df

,symptom,relation,symptom_short
0,uti,"Yes, A (uti) causes B (sepsis).",yes
1,low oxygen,"Yes, A (low oxygen) causes B (sepsis). Low oxy...",yes
2,low neutrophils,"Yes, A (low neutrophils) causes B (sepsis). Lo...",yes
3,low BP,"No. Sepsis can cause low blood pressure (BP), ...",no
4,high BP,"Yes, A (high BP) can cause B (sepsis). High bl...",yes
5,age>50,"Yes, A (age > 50) causes B (sepsis). As people...",yes
6,pregnancy,"No. Sepsis can be caused by various factors, i...",no
7,age>70,"Yes, A (age > 70) causes B (sepsis). As people...",yes


# Identify Comorbidities with Sepsis

For patients with sepsis, we idenify which other diseases they are likely to be diagnosed with. Then we identify whether these diseases are attributed to causing sepsis. If they are, then we use these as the input disease, to identify the biological indicators relating to these initial causal diseases. 

In [110]:
prompt_top_indicators = '''
[INST]
You are a medical researcher with expertise in diagnosing diseases. 
Your task is to identify what biological markers, demographic data, or symptoms are most crucial for diagnosing a specific disease.

Input Disease: [Disease_Name]

Please list the top medical tests, markers, and parameters that are essential for diagnosing [Disease_Name]. Please keep only the most crucial indicators. 

Return only a list of the biological indicators as your response to my query. The list should be in the following format:

['indicator_1', 'indicator_2', 'indicator_3', ... , 'indicator_n']

Input
{test}
[/INST]
'''

In [ ]:
run(prompt_top_indicators,"Input Disease: uti")

# Load data

In [15]:
lab_path = '../data/LABEVENTS.csv'
df_lab = pd.read_csv(lab_path)

lab_desc_path = '../data/D_LABITEMS.csv'
df_lab_desc = pd.read_csv(lab_desc_path)

df_items_desc = pd.read_csv('../data/D_ITEMS.csv')

In [9]:
df_items_desc[df_items_desc['ITEMID'].isin([618, 220210, 3603, 224689, 614, 651, 224422, 615, 224690])]


,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
259,571,614,Resp Rate (Spont),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
260,572,615,Resp Rate (Total),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
263,575,618,Respiratory Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1423,604,651,Spon RR (Mech.),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3857,2780,3603,Resp Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10053,13245,224689,Respiratory Rate (spontaneous),Respiratory Rate (spontaneous),metavision,chartevents,Respiratory,insp/min,Numeric,NaN
10054,13246,224690,Respiratory Rate (Total),Respiratory Rate (Total),metavision,chartevents,Respiratory,insp/min,Numeric,NaN
10413,13339,224422,Spont RR,Spont RR,metavision,chartevents,Respiratory,bpm,Numeric,NaN
11524,12738,220210,Respiratory Rate,RR,metavision,chartevents,Respiratory,insp/min,Numeric,NaN


# Microbiology events

In [9]:
# filter for only microbiology events
df_mb_desc = df_items_desc[df_items_desc['LINKSTO']=='microbiologyevents']

In [25]:
print("MB values:", len(df_mb_desc['LABEL'].values))
print("MB unique values:", len(df_mb_desc['LABEL'].unique()))

MB values: 436
MB unique values: 421


In [ ]:
df_mb_desc[df_mb_desc.duplicated(subset='LABEL', keep=False)]['LABEL'].unique()

In [112]:
df_mb_desc[df_mb_desc.duplicated(subset='LABEL', keep=False)].head()

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
10341,12340,80001,NaN,NaN,hospital,microbiologyevents,ORGANISM,NaN,NaN,NaN
10607,12247,70001,NaN,NaN,hospital,microbiologyevents,SPECIMEN,NaN,NaN,NaN
10638,12278,70032,Blood (EBV),NaN,hospital,microbiologyevents,SPECIMEN,NaN,NaN,NaN
10641,12281,70035,"FLUID,OTHER",NaN,hospital,microbiologyevents,SPECIMEN,NaN,NaN,NaN
10646,12286,70040,SWAB,NaN,hospital,microbiologyevents,SPECIMEN,NaN,NaN,NaN


# References

## Ref001
https://www.sciencedirect.com/science/article/pii/S2352914823000783#tbl1
https://doi.org/10.1016/j.imu.2023.101236

"Derived" means that the variable is calculated from other variables.

In [193]:
data_ref001 = {
    'Predictor': [
        'ICULOS', 'HospAdmTime', 'Age', 'Gender',
        'O2Sat', 'Resp', 'HR', 'Temp', 'DBP', 'MAP', 'SBP',
        'FiO2', 'pH', 'SIRS_Score', 'Base Excess', 'Glucose (serum)', 'PCO2', 'WBC', 'OXYGEN SATURATION',
        'Potassium', 'Calcium', 'Hematocrit', 'Hemoglobin', 'Magnesium', 'blood urea nitrogen', 'Phosphate', 'Bicarbonate',
        'Creatinine', 'PLATELET COUNT', 'Chloride'
    ],
    'Description': [
        'ICU length of stay (hours since ICU admission)', 'The time between hospital and ICU admission', 'Age (years)', 'Female (0) or male (1)',
        'Pulse oximetry (%)', 'Respiration rate (breaths per minute)', 'Heart rate (beats per minute)', 'Temperature (deg C)',
        'Diastolic BP (mm Hg)', 'Mean arterial pressure (mm Hg)', 'Systolic BP (mm Hg)',
        'Fraction of inspired oxygen (%)', 'A blood pH test is a normal part of a blood gas test or arterial blood gas (ABG) test. It measures how much oxygen and carbon dioxide are in your blood',
        'Four SIRS criteria were defined, namely tachycardia (heart rate >90 beats/min), tachypnea (respiratory rate >20 breaths/min), fever or hypothermia (temperature >38 or <36 °C), and leukocytosis, leukopenia, or bandemia (white blood cells >1200/mm3, <4000/mm3 or bandemia ≥10%',
        'Excess bicarbonate (mmol/L)', 'Serum glucose (mg/dL)', 'The partial pressure of carbon dioxide from arterial blood (mm Hg). Ranges 35 to 45 mmHg, or 4.7 to 6.0 kPa.', 'Leukocyte count (count/L)',
        'Oxygen saturation from arterial blood (%) SaO2', 'Potassiam (mmol/L)', 'Calcium (mg/dL)', 'Hematocrit (%)', 'Hemoglobin (g/dL)',
        'Magnesium (mmol/dL)', 'Blood urea nitrogen (mg/dL)', 'Phosphate (mg/dL)', 'Bicarbonate (mmol/L)', 'Creatinine (mg/dL)',
        'Platelet count (count/mL)', 'Chloride (mmol/L)'
    ],
    'Category': [
        'Demographic data', 'Demographic data', 'Demographic data', 'Demographic data',
        'Clinical time series data', 'Clinical time series data', 'Clinical time series data', 'Clinical time series data',
        'Clinical time series data', 'Clinical time series data', 'Clinical time series data', 
        'Laboratory values', 'Laboratory values', 'Laboratory values', 'Laboratory values', 'Laboratory values',
        'Laboratory values', 'Laboratory values', 'Laboratory values', 'Laboratory values', 'Laboratory values',
        'Laboratory values', 'Laboratory values', 'Laboratory values', 'Laboratory values', 'Laboratory values',
        'Laboratory values', 'Laboratory values', 'Laboratory values', 'Laboratory values'
    ],
    'Type': [
        'Numeric', 'Date', 'Numeric', 'Categorical',
        'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric',
        'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric',
        'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric',
        'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Numeric'
    ],
    'Table': [
        'ICUSTAYS', 'ADMISSIONS', 'PATIENTS', 'PATIENTS',
        'CHARTEVENTS', 'CHARTEVENTS', 'CHARTEVENTS', 'CHARTEVENTS', 'CHARTEVENTS', 'CHARTEVENTS', 'CHARTEVENTS',
        'CHARTEVENTS', 'LABEVENTS', 'Derived', 'LABEVENTS', 'CHARTEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS',
        'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS',
        'LABEVENTS', 'LABEVENTS', 'LABEVENTS'
    ],
    'ITEMID':[
        '', '', '', '', 
        '[646, 220277]', '[618, 220210, 3603, 224689, 614, 651, 224422, 615, 224690]', '[211, 220045]', '', '', '', '',
        '', '50820', '', '', '220621', '', '', '',
        '', '', '', '', '', '', '', '',
        '50912', '', ''
        
    ]
}

df_ref001 = pd.DataFrame(data_ref001)
df_ref001

,Predictor,Description,Category,Type,Table,ITEMID
0,ICULOS,ICU length of stay (hours since ICU admission),Demographic data,Numeric,ICUSTAYS,
1,HospAdmTime,The time between hospital and ICU admission,Demographic data,Date,ADMISSIONS,
2,Age,Age (years),Demographic data,Numeric,PATIENTS,
3,Gender,Female (0) or male (1),Demographic data,Categorical,PATIENTS,
4,O2Sat,Pulse oximetry (%),Clinical time series data,Numeric,CHARTEVENTS,"[646, 220277]"
5,Resp,Respiration rate (breaths per minute),Clinical time series data,Numeric,CHARTEVENTS,"[618, 220210, 3603, 224689, 614, 651, 224422, ..."
6,HR,Heart rate (beats per minute),Clinical time series data,Numeric,CHARTEVENTS,"[211, 220045]"
7,Temp,Temperature (deg C),Clinical time series data,Numeric,CHARTEVENTS,
8,DBP,Diastolic BP (mm Hg),Clinical time series data,Numeric,CHARTEVENTS,
9,MAP,Mean arterial pressure (mm Hg),Clinical time series data,Numeric,CHARTEVENTS,


Certain labels from the literature appear to not exist in the given data labels, however they may be coded differently. These labels were identified and manually converted to match the label in the given data. 

For labels that exist, there are often multiple associated ITEMID values; the test label depends on the fluid and category. We will need to confirm that the label, from the fluid and category, is important.

In [44]:
# 4 values of pH
df_lab_desc[df_lab_desc['LABEL'] == ('PH')]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
147,21,50820,pH,Blood,Blood Gas,11558-4
158,32,50831,pH,Other Body Fluid,Blood Gas,2748-2
420,294,51094,pH,Urine,Chemistry,2756-5
690,691,51491,pH,Urine,Hematology,5803-2


In [299]:
# multiple labels containing 'Platelet'
df_lab_desc[df_lab_desc['LABEL'].str.contains('Platelet', case=False)]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE,Exists,ref002
566,440,51240,LARGE PLATELETS,Blood,Hematology,34167-7,yes,yes
590,464,51264,PLATELET CLUMPS,Blood,Hematology,40741-1,yes,yes
591,465,51265,PLATELET COUNT,Blood,Hematology,777-3,yes,yes
592,466,51266,PLATELET SMEAR,Blood,Hematology,778-1,yes,yes


## Ref 002

Sup_data_Development and Evaluation of a Machine Learning Model for the Early Identification of Patients at Risk for Sepsis.

Table 1

In [192]:
data_ref002 = {
    'Predictor': [
        'AGE', 'DOBUTAMINE  (Y/N)', 'DOPAMINE  (Y/N)', 'EPINEPHRINE  (Y/N)', 'NOREPINEPHRINE  (Y/N)',
        'SYSTOLIC BP x AGE', 'SHOCK INDEX (HR/SYSTOLIC BP)', 'SHOCK INDEX x AGE',
        'ANION GAP', 'ALBUMIN', 'ALKALINE PHOSPHATASE', 'ASPARATE AMINOTRANSFERASE', 'BANDS', 'BILIRUBIN',
        'BLOOD UREA NITROGEN', 'CALCIUM', 'CREATININE', 'ESTIMATED GFR', 'GLUCOSE', 'LACTATE',
        'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHILS', 'PLATELET COUNT', 'WBC',
        'GLASGOW COMA SCALE', 'SUPPLEMENTAL OXYGEN (Y/N)', 'VENTILATOR', 'ALTERED MENTAL STATUS',
        'ABSCESS', 'ACUTE', 'ALTERED', 'BACTEREMIA', 'CELLULITIS', 'CYSTITIS', 'DIABETES',
        'FAILURE', 'LACTIC', 'LEUKOCYTOSIS', 'PNA', 'PNEUMONIA', 'PYELONEPHRITIS', 'RESPIRATORY',
        'SEPSIS', 'SEPTIC', 'UROSEPSIS', 'UTI',
        'HEART RATE', 'MEAN ARTERIAL PRESSURE', 'RESPIRATORY RATE', 'SYSTTOLIC BLOOD PRESSURE',
        'OXYGEN SATURATION', 'TEMPERATURE', 'WEIGHT', 'HEIGHT', 'BMI'
    ],
    'Category': [
        'Demographic', 'Medication', 'Medication', 'Medication', 'Medication',
        'Engineered Feature', 'Engineered', 'Engineered',
        'Laboratory Result', 'Laboratory Result', 'Laboratory Result', 'Laboratory Result', 'Laboratory Result', 'Laboratory Result',
        'Laboratory Result', 'Laboratory Result', 'Laboratory Result', 'Laboratory Result', 'Laboratory Result',
        'Laboratory Result', 'Laboratory Result', 'Laboratory Result', 'Laboratory Result', 'Laboratory Result', 'Laboratory Result',
        'Neurological Evaluation', 'Nursing Documentation', 'Nursing Documentation', 'Nursing Documentation',
        'Text- Keyword (ED chief complaint)', 'Text- Keyword (ED chief complaint)', 'Text- Keyword (ED chief complaint)', 'Text- Keyword (ED chief complaint)', 'Text- Keyword (ED chief complaint)', 'Text- Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)',
        'Text-Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)',
        'Text-Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)', 'Text-Keyword (ED chief complaint)',
        'Vital Sign', 'Vital Sign', 'Vital Sign', 'Vital Sign',
        'Vital Sign', 'Vital Sign', 'Physiological', 'Physiological', 'Physiological'
    ],
    'Table': [
        'PATIENTS', 'PRESCRIPTIONS', 'PRESCRIPTIONS', 'PRESCRIPTIONS', 'PRESCRIPTIONS',
        'Derived', 'Derived', 'Derived',
        'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS',
        'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS',
        'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS', 'LABEVENTS',
        'CHARTEVENTS', 'CHARTEVENTS', 'CHARTEVENTS', 'CHARTEVENTS',
        'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS',
        'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS',
        'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS', 'ED_EVENTS',
        'CHARTEVENTS', 'CHARTEVENTS', 'CHARTEVENTS', 'CHARTEVENTS',
        'CHARTEVENTS', 'CHARTEVENTS', 'ADMISSIONS', 'ADMISSIONS', 'Derived'
    ]
}

df_ref002 = pd.DataFrame(data_ref002)
df_ref002

,Predictor,Category,Table
0,AGE,Demographic,PATIENTS
1,DOBUTAMINE (Y/N),Medication,PRESCRIPTIONS
2,DOPAMINE (Y/N),Medication,PRESCRIPTIONS
3,EPINEPHRINE (Y/N),Medication,PRESCRIPTIONS
4,NOREPINEPHRINE (Y/N),Medication,PRESCRIPTIONS
5,SYSTOLIC BP x AGE,Engineered Feature,Derived
6,SHOCK INDEX (HR/SYSTOLIC BP),Engineered,Derived
7,SHOCK INDEX x AGE,Engineered,Derived
8,ANION GAP,Laboratory Result,LABEVENTS
9,ALBUMIN,Laboratory Result,LABEVENTS


## Ref003

In [191]:
data_ref003 = {
'Predictor': [
    'Lactate',
    'Systolic BP', 
    'Diastolic BP',
    'Glasgow Coma Scale',
    'Heart Rate',
    'Respiratory Rate',
    'WBC',
    'WBC',
    'C-Reactive Protein',
    'Blood Culture',
    'Temperature',
    'Platelet Count',
    'Bilirubin',
    'Creatinine',
    'Bicarbonate',
    'Blood Glucose',
    'Urine Culture',
    'ASPARATE AMINOTRANSFERASE', 
    'ALANINE AMINOTRANSFERASE',
    'Troponin',
    'Hemoglobin',
    'INR',
    'PTT',
    'Albumin',
    'D-Dimer',
    'Ferritin',
    'LACTATE DEHYDROGENASE',
    'BLOOD UREA NITROGEN',
    'PaO2', 
    'FiO2', 
    'Age',
    'Comorbidities',
    'Surgery History',
    'Immunosuppression Status',
    'Alcohol Abuse History',
    'Intravenous Drug Use History',
    'Recent Infection or Antibiotic Use',
    'Mechanical Ventilation Status',
    'Central Venous Catheter Use',
    'Urinary Catheter Use',
    'Recent Hospitalization',
    'Chronic Renal Failure',
    'Chronic Liver Disease',
    'Pregnancy Status',
    'Burns or Trauma',
    'NEUTROPHILS',
    'Nutritional Status',
    'BMI',
    'Infection Source',
    'Skin Mottling',
    'Skin Color',
    'Urine Output'
],
'Description': [
    "High Lactate Levels",
    "Low Blood Pressure (Hypotension) ('Systolic BP', 'Diastolic BP')",
    "Low Blood Pressure (Hypotension) ('Systolic BP', 'Diastolic BP')",
    "Altered Mental Status",
    "High Heart Rate (Tachycardia)",
    "Rapid Breathing (Tachypnea)",
    "High WBC Count",
    "Low WBC Count",
    "Elevated CRP",
    "Positive Blood Culture",
    "Fever or Hypothermia",
    "Low Platelet Count",
    "Elevated Bilirubin",
    "Elevated Creatinine",
    "Low Bicarbonate Levels",
    "High Blood Glucose",
    "Positive Urine Culture",
    "Elevated Liver Enzymes (AST, ALT)",
    "Elevated Liver Enzymes (AST, ALT)",
    "Elevated Troponin",
    "Low Hemoglobin",
    "High INR or PTT",
    "High INR or PTT",
    "Low Albumin",
    "Elevated D-Dimer",
    "Elevated Ferritin",
    "Elevated LDH (LACTATE DEHYDROGENASE)",
    "High Blood Urea Nitrogen",
    "Low PaO2/FiO2 Ratio (Partial pressure of oxygen)",
    "Low PaO2/FiO2 Ratio (fraction of inspired oxygen)",
    "Age > 65 years",
    "Comorbidities (e.g., Diabetes, Heart Disease)",
    "Recent Surgery",
    "Immunosuppression",
    "Chronic Alcohol Abuse",
    "Intravenous Drug Use",
    "Recent Infection or Antibiotic Use",
    "Mechanical Ventilation",
    "Central Venous Catheter Use",
    "Urinary Catheter Use",
    "Recent Hospitalization",
    "Chronic Renal Failure",
    "Chronic Liver Disease",
    "Pregnancy",
    "Burns or Trauma",
    "Neutropenia",
    "Malnutrition",
    "Obesity",
    "Community-Acquired vs Hospital-Acquired Infection",
    "Skin Mottling",
    "Cyanosis (Bluish Skin)",
    "Oliguria (Low Urine Output)"
],
'Table': [
    "labevents",  # High Lactate Levels
    "chartevents",  # Low Blood Pressure (Hypotension)
    "chartevents",  # Low Blood Pressure (Hypotension)
    "chartevents",  # Altered Mental Status
    "chartevents",  # High Heart Rate (Tachycardia)
    "chartevents",  # Rapid Breathing (Tachypnea)
    "labevents",  # High WBC Count
    "labevents",  # Low WBC Count
    "labevents",  # Elevated CRP
    "microbiologyevents",  # Positive Blood Culture
    "chartevents",  # Fever or Hypothermia
    "labevents",  # Low Platelet Count
    "labevents",  # Elevated Bilirubin
    "labevents",  # Elevated Creatinine
    "labevents",  # Low Bicarbonate Levels
    "labevents",  # High Blood Glucose
    "microbiologyevents",  # Positive Urine Culture
    "labevents",  # Elevated Liver Enzymes (AST, ALT)
    "labevents",  # Elevated Liver Enzymes (AST, ALT)
    "labevents",  # Elevated Troponin
    "labevents",  # Low Hemoglobin
    "labevents",  # High INR or PTT
    "labevents",  # High INR or PTT
    "labevents",  # Low Albumin
    "labevents",  # Elevated D-Dimer
    "labevents",  # Elevated Ferritin
    "labevents",  # Elevated LDH
    "labevents",  # High BUN
    "chartevents",  # Low PaO2/FiO2 Ratio
    "chartevents",  # Low PaO2/FiO2 Ratio (fraction of inspired oxygen)
    "patients",  # Age > 65 years
    "admissions",  # Comorbidities (e.g., Diabetes, Heart Disease)
    "admissions",  # Recent Surgery
    "admissions",  # Immunosuppression
    "admissions",  # Chronic Alcohol Abuse
    "admissions",  # Intravenous Drug Use
    "admissions",  # Recent Infection or Antibiotic Use
    "chartevents",  # Mechanical Ventilation
    "procedures_icd",  # Central Venous Catheter Use
    "procedures_icd",  # Urinary Catheter Use
    "admissions",  # Recent Hospitalization
    "admissions",  # Chronic Renal Failure
    "admissions",  # Chronic Liver Disease
    "admissions",  # Pregnancy
    "admissions",  # Burns or Trauma
    "labevents",  # Neutropenia
    "admissions",  # Malnutrition
    "admissions",  # Obesity
    "admissions",  # Community-Acquired vs Hospital-Acquired Infection
    "chartevents",  # Skin Mottling
    "chartevents",  # Cyanosis (Bluish Skin)
    "outputevents"  # Oliguria (Low Urine Output)
]
}

df_ref003 = pd.DataFrame(data_ref003)
df_ref003['Table'] = df_ref003['Table'].str.upper()
df_ref003

,Predictor,Description,Table
0,Lactate,High Lactate Levels,LABEVENTS
1,Systolic BP,Low Blood Pressure (Hypotension) ('Systolic BP...,CHARTEVENTS
2,Diastolic BP,Low Blood Pressure (Hypotension) ('Systolic BP...,CHARTEVENTS
3,Glasgow Coma Scale,Altered Mental Status,CHARTEVENTS
4,Heart Rate,High Heart Rate (Tachycardia),CHARTEVENTS
5,Respiratory Rate,Rapid Breathing (Tachypnea),CHARTEVENTS
6,WBC,High WBC Count,LABEVENTS
7,WBC,Low WBC Count,LABEVENTS
8,C-Reactive Protein,Elevated CRP,LABEVENTS
9,Blood Culture,Positive Blood Culture,MICROBIOLOGYEVENTS


# Save references to CSV

In [61]:
# Store dataframes to CSV

from pathlib import Path
ROOT_DIR = Path('..')

reference_dataframes = [df_ref001, df_ref002, df_ref003]

# Create the directory if it doesn't exist
reference_directory = ROOT_DIR / 'data' / 'reference'
reference_directory.mkdir(parents=True, exist_ok=True)

for i in range(len(reference_dataframes)):
  ref_name = 'ref_00' + str(i + 1) + '.csv'
  # Save the DataFrames to a CSV file
  csv_file_path = Path(ROOT_DIR / 'data' / 'reference' / ref_name )
  reference_dataframes[i].to_csv(csv_file_path, index=False)

In [37]:
df_ref003[df_ref003['Table']=='labevents']
df_ref002
df_lab_desc[df_lab_desc['ITEMID']==50816]
df_lab_desc[df_lab_desc['ITEMID']==50886]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
213,87,50886,BLOOD CULTURE HOLD,BLOOD,CHEMISTRY,NaN


In [ ]:
# load all from directory
import os
csv_files = [f for f in os.listdir(reference_directory) if f.endswith('.csv')]

dfs = []  # To store the DataFrames
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    dfs.append(pd.read_csv(file_path)

In [ ]:
prompt_check_lab_events = '''
[INST]
You are a medical expert. You are tasked to identify causality between two conditions. 
Return yes if A is important in identifying B. If B causes A or if you are not sure, return no. 
Keep your responses short and succinct. 
Input
{test}
[/INST]
'''
df['relation'] = df.apply(lambda x: run(prompt_causality, f"A: {x['LABEL']} with fluid type of {x['FLUID']}, B: sepsis"), axis=1)

In [39]:
# Convert all labevent strings to uppercase 
df_lab_desc = df_lab_desc.applymap(lambda x: x.upper() if type(x) == str else x)
  
def get_filtered_reference(table_query, reference):
  """
    Parameters: 
    - table_query: the data table name (LABEVENTS, microbiology, CHARTEVENTS, ADMISSIONS)
    - reference [DataFrame]: Sepsis reference with the columns 'Predictors' and 'Table'
    
    Returns:
    - reference_filtered [DataFrame]: The reference, filtered by the table query. The column 'Exists' represents the predictor's presence in the data.
    - predictors_missing [array]: Predictors not in the data. 
    
  """
  # convert reference string data to uppercase
  reference = reference.applymap(lambda x: x.upper() if type(x) == str else x)
  
  # filter by name of table (input)
  reference_filtered = reference[reference['Table'] == table_query]

  # check if variable exists in table input data
  reference_filtered['Exists'] = reference_filtered['Predictor'].apply(lambda x: 'yes' if x in df_lab_desc['LABEL'].values else 'no')
  
  # debugging to check the predictors not in df
  predictors_missing = reference_filtered[reference_filtered['Exists']=='no']['Predictor'].values
  
  return reference_filtered, predictors_missing


/var/folders/88/v73j8qys0vjg1tz4wb20c2kh0000gn/T/ipykernel_1136/428479399.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lab_desc = df_lab_desc.applymap(lambda x: x.upper() if type(x) == str else x)


# Filter References by Labevents

In [194]:
df_ref001_lab, df_ref001_lab_missing = get_filtered_reference('LABEVENTS', df_ref001)
df_ref002_lab, df_ref002_lab_missing = get_filtered_reference('LABEVENTS', df_ref002)
df_ref003_lab, df_ref003_lab_missing = get_filtered_reference('LABEVENTS', df_ref003)

In [195]:
print(df_ref001_lab_missing)
print(df_ref002_lab_missing)
print(df_ref003_lab_missing)

['CALCIUM' 'BLOOD UREA NITROGEN']
['ASPARATE AMINOTRANSFERASE' 'BLOOD UREA NITROGEN' 'CALCIUM'
 'ESTIMATED GFR' 'LYMPHOCYTE' 'MONOCYTE']
['BLOOD GLUCOSE' 'ASPARATE AMINOTRANSFERASE' 'ALANINE AMINOTRANSFERASE'
 'TROPONIN' 'INR' 'LACTATE DEHYDROGENASE' 'BLOOD UREA NITROGEN']


In [125]:
df_ref001_lab

,Predictor,Description,Category,Type,Table,ITEMID,Exists,Matching_ITEMID
12,PH,A BLOOD PH TEST IS A NORMAL PART OF A BLOOD GA...,LABORATORY VALUES,NUMERIC,LABEVENTS,,yes,"[50820, 50831, 51094, 51491]"
14,BASE EXCESS,EXCESS BICARBONATE (MMOL/L),LABORATORY VALUES,NUMERIC,LABEVENTS,,yes,[50802]
15,GLUCOSE,SERUM GLUCOSE (MG/DL),LABORATORY VALUES,NUMERIC,LABEVENTS,,yes,"[50809, 50842, 50931, 51014, 51022, 51034, 510..."
16,PCO2,THE PARTIAL PRESSURE OF CARBON DIOXIDE FROM AR...,LABORATORY VALUES,NUMERIC,LABEVENTS,,yes,"[50818, 50830]"
17,WBC,LEUKOCYTE COUNT (COUNT/L),LABORATORY VALUES,NUMERIC,LABEVENTS,,yes,"[51363, 51384, 51439, 51458, 51128, 51300, 515..."
18,OXYGEN SATURATION,OXYGEN SATURATION FROM ARTERIAL BLOOD (%) SAO2,LABORATORY VALUES,NUMERIC,LABEVENTS,,yes,[50817]
19,POTASSIUM,POTASSIAM (MMOL/L),LABORATORY VALUES,NUMERIC,LABEVENTS,,yes,"[50822, 50833, 50847, 50971, 51041, 51057, 510..."
20,CALCIUM,CALCIUM (MG/DL),LABORATORY VALUES,NUMERIC,LABEVENTS,,no,"[51468, 51469, 51470, 50808, 50893, 51029, 510..."
21,HEMATOCRIT,HEMATOCRIT (%),LABORATORY VALUES,NUMERIC,LABEVENTS,,yes,"[51348, 51369, 51422, 51445, 50810, 51115, 512..."
22,HEMOGLOBIN,HEMOGLOBIN (G/DL),LABORATORY VALUES,NUMERIC,LABEVENTS,,yes,"[50805, 50811, 50814, 50852, 50855, 51212, 512..."


In [124]:
df_ref002_lab

,Predictor,Category,Table,Exists,Matching_ITEMID
8,ANION GAP,LABORATORY RESULT,LABEVENTS,yes,[50868]
9,ALBUMIN,LABORATORY RESULT,LABEVENTS,yes,"[50835, 50862, 51011, 51019, 51025, 51046, 510..."
10,ALKALINE PHOSPHATASE,LABORATORY RESULT,LABEVENTS,yes,[50863]
11,ASPARATE AMINOTRANSFERASE,LABORATORY RESULT,LABEVENTS,no,[50878]
12,BANDS,LABORATORY RESULT,LABEVENTS,yes,"[51366, 51386, 51441, 51111, 51144, 51344]"
13,BILIRUBIN,LABORATORY RESULT,LABEVENTS,yes,"[51464, 51465, 50838, 50883, 50884, 50885, 510..."
14,UREA NITROGEN,LABORATORY RESULT,LABEVENTS,yes,"[50851, 51006, 51045, 51104]"
15,CALCIUM,LABORATORY RESULT,LABEVENTS,no,"[51468, 51469, 51470, 50808, 50893, 51029, 510..."
16,CREATININE,LABORATORY RESULT,LABEVENTS,yes,"[50841, 50912, 51021, 51032, 51052, 51067, 510..."
17,ESTIMATED GFR,LABORATORY RESULT,LABEVENTS,no,[50920]


In [123]:
df_ref003_lab

,Predictor,Description,Table,Exists,Matching_ITEMID
0,LACTATE,HIGH LACTATE LEVELS,LABEVENTS,yes,"[50813, 50843, 50954, 51015, 51054]"
6,WBC,HIGH WBC COUNT,LABEVENTS,yes,"[51363, 51384, 51439, 51458, 51128, 51300, 515..."
7,WBC,LOW WBC COUNT,LABEVENTS,yes,"[51363, 51384, 51439, 51458, 51128, 51300, 515..."
8,C-REACTIVE PROTEIN,ELEVATED CRP,LABEVENTS,yes,[50889]
11,PLATELET COUNT,LOW PLATELET COUNT,LABEVENTS,yes,[51265]
12,BILIRUBIN,ELEVATED BILIRUBIN,LABEVENTS,yes,"[51464, 51465, 50838, 50883, 50884, 50885, 510..."
13,CREATININE,ELEVATED CREATININE,LABEVENTS,yes,"[50841, 50912, 51021, 51032, 51052, 51067, 510..."
14,BICARBONATE,LOW BICARBONATE LEVELS,LABEVENTS,yes,"[50803, 50837, 50882, 51027, 51048, 51061, 51076]"
15,BLOOD GLUCOSE,HIGH BLOOD GLUCOSE,LABEVENTS,no,None
17,ASPARATE AMINOTRANSFERASE,"ELEVATED LIVER ENZYMES (AST, ALT)",LABEVENTS,no,[50878]


In [ ]:


suspicious_predictors = ['LYMPHOCYTE', 'MONOCYTE', 'PLATE', 'NEUTROPHIL', 'CALCIUM', 'LACTATE', 'AMINOTRANSFERASE', 'ALBUMIN']

def verify_labels(df, suspicious_predictors):
    filtered_dfs = []
    for predictor in suspicious_predictors:
        filtered_df = df_lab_desc[df_lab_desc['LABEL'].apply(lambda x: word in x)]
        filtered_dfs.append(filtered_df)
    df_query_labels = pd.concat(filtered_dfs).drop_duplicates()
    return df_query_labels
print("Words related to ref002:")
print("Input:", len(suspicious_predictors))
print("Related labels:", checkwords(df_lab_desc, suspicious_predictors).shape[0])
checkwords(df_lab_desc, suspicious_predictors)

In [202]:
verified_ids = {
    'PH': 50820, 
    'NEUTROPHILS': 51256,
    'ALKALINE PHOSPHATASE': 50863,
    'CREATININE': 50912,
    'BANDS': 51144,
    'BILIRUBIN': 50885,
    'HEMATOCRIT': 51221,            # total
    'LACTATE DEHYDROGENASE': 50954,  # (LD)
    'LACTATE': 50813,
    'WBC': 51301,
    'BLOOD UREA NITROGEN': 51006, # blood urea nitrogen
    'MONOCYTES': 51254
    
}

potential_ids = {
    'HEMOGLOBIN':[51222, 50811, 50852, 50855],    # seem to relate to the same thing
    'D-DIMER': [51196, 50915],                    # both from blood
    'BICARBONATE': [50882, 50803],                # blood and total
    'Calcium': [50893, 50808],
    'WBC': [51300, 51301],                        # same, from blood
    'BLOOD GLUCOSE': [50809, 50931, 51529],
    'ALBUMIN': [50862, 51069, 51070],
    'LYMPHOCYTE': [51244, 51143, 51245]
}




# Function to match 'Predictor' to 'LABEL' and return 'ITEMID'
def match_predictor_to_itemid(predictor):
    
    if predictor in verified_ids.keys():
        # Find ITEMID from exclude_search dictionary
        itemid = verified_ids[predictor]
        
        # Check for an exact match when the predictor is in the exclude_search list
        exact_match = df_lab_desc[df_lab_desc['ITEMID'] == itemid]
        if not exact_match.empty:
            return exact_match['ITEMID'].tolist()
        return None
    
    else:
        # Check for non-exact match.    
        matching_rows = df_lab_desc[df_lab_desc['LABEL'].str.contains(predictor, case=False, na=False)]
        if matching_rows.empty:
            return None
        else:
            return matching_rows['ITEMID'].tolist()

In [200]:
df_ref003_lab


,Predictor,Description,Table,Exists,Matching_ITEMID
0,LACTATE,HIGH LACTATE LEVELS,LABEVENTS,yes,[50813]
6,WBC,HIGH WBC COUNT,LABEVENTS,yes,[51301]
7,WBC,LOW WBC COUNT,LABEVENTS,yes,[51301]
8,C-REACTIVE PROTEIN,ELEVATED CRP,LABEVENTS,yes,[50889]
11,PLATELET COUNT,LOW PLATELET COUNT,LABEVENTS,yes,[51265]
12,BILIRUBIN,ELEVATED BILIRUBIN,LABEVENTS,yes,[50885]
13,CREATININE,ELEVATED CREATININE,LABEVENTS,yes,[50912]
14,BICARBONATE,LOW BICARBONATE LEVELS,LABEVENTS,yes,"[50803, 50837, 50882, 51027, 51048, 51061, 51076]"
15,BLOOD GLUCOSE,HIGH BLOOD GLUCOSE,LABEVENTS,no,None
17,ASPARATE AMINOTRANSFERASE,"ELEVATED LIVER ENZYMES (AST, ALT)",LABEVENTS,no,[50878]


In [ ]:
# Apply the function to the 'Predictor' column and create a new column called 'Matching_ITEMID'
df_ref001_lab['Matching_ITEMID'] = df_ref001_lab['Predictor'].apply(match_predictor_to_itemid)
df_ref002_lab['Matching_ITEMID'] = df_ref002_lab['Predictor'].apply(match_predictor_to_itemid)
df_ref003_lab['Matching_ITEMID'] = df_ref003_lab['Predictor'].apply(match_predictor_to_itemid)

# Add to original data

In [198]:
references = [df_ref001, df_ref002, df_ref003]

table_query = "LABEVENTS"
refcount = 0

# Function to check if any 'Predictor' exists in the 'LABEL'
def check_label_in_predictor(label):
  for predictor in df_reference['Predictor']:
    if predictor in exclude_search:
      if predictor == label:
        return 'yes'
      continue
    if predictor in label:
      # todo: match only word
      return 'yes'
  return 'no'


for reference in references: 
  # get unique reference name
  refcount+=1
  reference.name = "ref" + str(refcount)
  
  # filter reference by table
  df_reference = get_filtered_reference(table_query, reference)[0]
  
  # add reference for each feature
  #df_lab_desc[reference.name] = df_lab_desc['LABEL'].apply(lambda x: 'yes' if any(feature in x for feature in df_reference['Predictor']) else 'no')
  
  
  # Apply the function to the 'LABEL' column and create a new column called 'reference_name_exists'
  df_lab_desc[reference.name] = df_lab_desc['LABEL'].apply(check_label_in_predictor)


In [199]:
df_potential_lab = df_lab_desc[(df_lab_desc['ref1']=='yes') | (df_lab_desc['ref2']=='yes') | (df_lab_desc['ref3']=='yes')]
df_potential_lab

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE,ref1,ref2,ref3
2,548,51348,"HEMATOCRIT, CSF",CEREBROSPINAL FLUID (CSF),HEMATOLOGY,30398-2,yes,no,no
9,555,51355,MONOCYTES,CEREBROSPINAL FLUID (CSF),HEMATOLOGY,26486-1,no,yes,no
17,563,51363,"WBC, CSF",CEREBROSPINAL FLUID (CSF),HEMATOLOGY,26465-5,yes,yes,yes
19,565,51365,ATYPICAL LYMPHOCYTES,JOINT FLUID,HEMATOLOGY,33371-6,no,yes,no
20,566,51366,BANDS,JOINT FLUID,HEMATOLOGY,33361-7,no,yes,no
...,...,...,...,...,...,...,...,...,...
716,717,51517,WBC CASTS,URINE,HEMATOLOGY,5820-6,yes,yes,yes
717,718,51518,WBC CLUMPS,URINE,HEMATOLOGY,NaN,yes,yes,yes
728,729,51529,ESTIMATED ACTUAL GLUCOSE,BLOOD,CHEMISTRY,NaN,no,yes,no
732,733,51533,WBCP,BLOOD,HEMATOLOGY,NaN,yes,yes,yes


# Save final CSV files

In [201]:
from pathlib import Path
ROOT_DIR = Path('..')

# Create the directory if it doesn't exist
reference_path = Path(ROOT_DIR / 'data' / 'reference')
reference_path.mkdir(parents=True, exist_ok=True)

reference_dataframes = [df_ref001_lab, df_ref002_lab, df_ref003_lab]

for i in range(len(reference_dataframes)):
  ref_name = 'ref_00' + str(i + 1) + '_labevents.csv'
  reference_dataframes[i].to_csv(Path(reference_path  / ref_name), index=False)

df_potential_lab.to_csv(Path(reference_path / "potential_labevents.csv"), index=False)

In [152]:
df_potential_lab

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE,ref1,ref2,ref3
2,548,51348,"HEMATOCRIT, CSF",CEREBROSPINAL FLUID (CSF),HEMATOLOGY,30398-2,yes,no,no
9,555,51355,MONOCYTES,CEREBROSPINAL FLUID (CSF),HEMATOLOGY,26486-1,no,yes,no
17,563,51363,"WBC, CSF",CEREBROSPINAL FLUID (CSF),HEMATOLOGY,26465-5,yes,yes,yes
19,565,51365,ATYPICAL LYMPHOCYTES,JOINT FLUID,HEMATOLOGY,33371-6,no,yes,no
20,566,51366,BANDS,JOINT FLUID,HEMATOLOGY,33361-7,no,yes,no
...,...,...,...,...,...,...,...,...,...
716,717,51517,WBC CASTS,URINE,HEMATOLOGY,5820-6,yes,yes,yes
717,718,51518,WBC CLUMPS,URINE,HEMATOLOGY,NaN,yes,yes,yes
728,729,51529,ESTIMATED ACTUAL GLUCOSE,BLOOD,CHEMISTRY,NaN,yes,yes,no
732,733,51533,WBCP,BLOOD,HEMATOLOGY,NaN,yes,yes,yes


# Verify Labels

In [159]:
df_lab_desc[df_lab_desc['LABEL'].str.contains('PLATE', case=False)]
# There are 4 items relating to platelets and neutrophils, 5 items related to monocytes, and 13 related to lymphocyctes. The data misspells the word 'ASPARATE'. We should also check for both AST and ALT. 

df_lab_desc[df_lab_desc['LABEL'].str.contains('MONOCYTE', case=False)]
# There are 5 monocytes. 
# We would likely be interested in those returned from Blood fluids
# 51253, 51254

df_lab_desc[df_lab_desc['LABEL'].str.contains('LYMPHOCYTE', case=False)]
# thirteen lymphocytes


df_lab_desc[df_lab_desc['LABEL'].str.contains('AMINOTRANSFERASE', case=False)]
#df_lab_desc['LABEL'] = df_lab_desc['LABEL'].replace('ASPARATE AMINOTRANSFERASE (AST)', 'ASPARTATE AMINOTRANSFERASE (AST)')

df_lab_desc[df_lab_desc['LABEL'].str.contains('CALCIUM')]
# ESTIMATED GFR (MDRD EQUATION) 


,ITEMID,LABEL,FLUID,ref1,ref2
122,51468,CALCIUM CARBONATE CRYSTALS,URINE,yes,yes
123,51469,CALCIUM OXALATE CRYSTALS,URINE,yes,yes
124,51470,CALCIUM PHOSPHATE CRYSTALS,URINE,yes,yes
130,50803,"CALCULATED BICARBONATE, WHOLE BLOOD",BLOOD,yes,no
131,50804,CALCULATED TOTAL CO2,BLOOD,no,no
135,50808,FREE CALCIUM,BLOOD,yes,yes
137,50810,"HEMATOCRIT, CALCULATED",BLOOD,yes,no
220,50893,"CALCIUM, TOTAL",BLOOD,yes,yes
221,50894,CALCULATED FREE TESTOSTERONE,BLOOD,no,no
222,50895,CALCULATED TBG,BLOOD,no,no


In [ ]:
# bloodglucose = [50809, 50931, 51529]
# LACTATE DEHYDROGENASE(LD) = [50954]

# need to be related only to the blood:
# PH = 50820
# CREATININE = 50912
# BANDS = 51144	

# BILIRUBIN potential ITEMID = [51028, 50883, 51012, 50838, 51049, 50885, 50884, 51464]
# BILIRUBIN = [50885]
# assuming Bilirubin Total

# Platelet Count = [51265]

# Lymphocytes = [51244, 51143, 51245]
# 51244	LYMPHOCYTES
# 51143	ATYPICAL LYMPHOCYTES	
# 51245	LYMPHOCYTES, PERCENT

# Calcium = [50893, 50808]
# assuming calcium is from blood
# 50893	CALCIUM, TOTAL		labevents	591999
# 50808	FREE CALCIUM		labevents	249111


# ALBUMIN
# 50862	ALBUMIN	BLOOD
# 51069	ALBUMIN, URINE	URINE
# 51070	ALBUMIN/CREATININE, URINE	URINE	

# BICARBONATE
# 50882	BICARBONATE		labevents	780733	CHEMISTRY
# 50803	CALCULATED BICARBONATE, WHOLE BLOOD		labevents	9246

# D-DIMER
# 51196	D-DIMER		labevents	4476	HEMATOLOGY		BLOOD
# 50915	D-DIMER		labevents	1192	CHEMISTRY		BLOOD


# LACTIC ACID 
# is calculated using LACTATE [50813]

# LACTIC ACID chartevents [225668, 1531]

# HEMOGLOBIN
# 51222	HEMOGLOBIN		labevents	752523	HEMATOLOGY		BLOOD
# 50811	HEMOGLOBIN		labevents	89712	BLOOD GAS		BLOOD
# 50852	% HEMOGLOBIN A1C		labevents	16618	CHEMISTRY		BLOOD
# 50855	ABSOLUTE HEMOGLOBIN		labevents	3791	CHEMISTRY		BLOOD
